In [1]:
!pip install pymysql
import pymysql

host='49.247.130.147'
username='root'
password='jihyun0127'
db_name='kpc_test'
db= pymysql.connect(host=host, user=username, passwd=password, db=db_name, connect_timeout=1)

     |████████████████████████████████| 51kB 2.3MB/s 


In [0]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://"+username+":"+password+"@"+host+":3306/"+db_name+"?charset=utf8", encoding='utf-8')
conn = engine.connect()

In [3]:
import pandas as pd

movie1 = pd.read_sql_table('att_table', conn)
movie2 = pd.read_sql_table('keyword_table',conn)
movies = pd.merge(movie1,movie2,on='id')
ratings = pd.read_sql_table('review_table', conn)
movies.head()

,id,name,star,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
0,320359,창덕궁,45,4610,T020303,"종로구 율곡로 99,",37.580095,126.991850,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/amazing/great/histo...
1,324887,덕수궁,40,1074,T020303,"중구 세종대로 99,",37.565871,126.975008,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,free,free/interesting/modern/traditional/nice/beaut...
2,324888,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/historical/great/in...
3,324891,롯데월드,40,3111,T030108,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,extra,worth,great/extra/young/fun/worth/big/huge/nice/mode...
4,324907,남대문시장,40,2894,T020204,"중구 남대문시장4길 21,",37.559147,126.977641,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,cheap,interesting/cheap/traditional/nice/great/expen...


In [4]:
movies

,id,name,star,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
0,320359,창덕궁,45,4610,T020303,"종로구 율곡로 99,",37.580095,126.991850,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/amazing/great/histo...
1,324887,덕수궁,40,1074,T020303,"중구 세종대로 99,",37.565871,126.975008,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,free,free/interesting/modern/traditional/nice/beaut...
2,324888,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/historical/great/in...
3,324891,롯데월드,40,3111,T030108,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,extra,worth,great/extra/young/fun/worth/big/huge/nice/mode...
4,324907,남대문시장,40,2894,T020204,"중구 남대문시장4길 21,",37.559147,126.977641,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,cheap,interesting/cheap/traditional/nice/great/expen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,12518697,쿠킹 난타,45,1881,T030207,"서대문구 충정로 7,",37.560133,126.962236,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,big,expensive,great/funny/enjoyable/young/energetic/interest...
124,15306289,서울식물원,45,26,T010101,"강서구 마곡동로 161,",37.569244,126.835378,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,huge,free,nice/free/interesting/huge/beautiful/peaceful/...
125,15816287,골든블루마리나 - 서초지점,50,1,T030114,"2085-14, Olympic-daero, Seocho-gu,",37.483712,127.032411,L0101,https://media-cdn.tripadvisor.com/media/photo-...,0,0,0,0
126,17124032,서울이스케이프룸 - 강남2호점,50,500,T030107,"강남구 강남대로102길 20,",37.502220,127.027294,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,big,worth,great/fun/nice/friendly/funny/unique/worth/won...


In [0]:
rating_movies = pd.merge(ratings, movies, on='id')

In [0]:
mname=list(movies['name'])

In [7]:
rating_movies

,review_num,id,user,star_x,month,review,name,star_y,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
0,1,1169465,Cyril Grace R,50,November,this place had 360 degree vies of seoul.best t...,N 서울 타워,40,9451,T020207,"용산구 남산공원길 105,",37.550947,126.989296,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,worthy,worth,nice/beautiful/amazing/great/fantastic/special...
1,4,1169465,Blossom,50,May,The Korean Drama that left the biggest impact ...,N 서울 타워,40,9451,T020207,"용산구 남산공원길 105,",37.550947,126.989296,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,worthy,worth,nice/beautiful/amazing/great/fantastic/special...
2,5,1169465,Kens,40,January,I went to Namsan on my first day at Seoul. I w...,N 서울 타워,40,9451,T020207,"용산구 남산공원길 105,",37.550947,126.989296,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,worthy,worth,nice/beautiful/amazing/great/fantastic/special...
3,8,1169465,Catherine Q,50,April,If your in Seoul you should visit the N Tower....,N 서울 타워,40,9451,T020207,"용산구 남산공원길 105,",37.550947,126.989296,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,worthy,worth,nice/beautiful/amazing/great/fantastic/special...
4,10,1169465,jrrjr619,40,February,The N Seoul Tower is relatively easy to get to...,N 서울 타워,40,9451,T020207,"용산구 남산공원길 105,",37.550947,126.989296,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,worthy,worth,nice/beautiful/amazing/great/fantastic/special...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4447,4362,5924606,BigBoyA77,50,July,So this is job themed play area. Kidzania is v...,키자니아 서울,40,31,T030113,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,special,huge,worth,special/nice/worth/friendly/young/archaeologic...
4448,4363,5924606,Joanne H,40,June,Nice place to go to.... my kiddo loved it. It ...,키자니아 서울,40,31,T030113,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,special,huge,worth,special/nice/worth/friendly/young/archaeologic...
4449,4364,5924606,EasyTravellerLam,40,March,"Went with my 2 boys, 8 and 5 years old. I thin...",키자니아 서울,40,31,T030113,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,special,huge,worth,special/nice/worth/friendly/young/archaeologic...
4450,4365,5924606,Francky,50,June,It's a place to visit when you have young kids...,키자니아 서울,40,31,T030113,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,special,huge,worth,special/nice/worth/friendly/young/archaeologic...


In [8]:
ratings = ratings[['user', 'id', 'star']]
ratings_matrix = ratings.pivot_table('star', index='user', columns='id')
ratings_matrix

id,320359,324887,324888,324891,324907,325043,502576,553213,553246,553546,554528,554533,554537,554551,554582,554634,561808,590748,592484,592486,592490,592495,592506,592508,592658,598499,609340,1046419,1169465,1235392,1379963,1489411,1519813,1549822,1551271,1552278,1554603,1579821,1604009,1643534,...,4084344,4084386,4094861,4095685,4096021,4427670,4798698,4798711,4798715,5924606,5924658,6509932,6656879,6671988,6847826,6847900,6960656,7028449,7060791,7264192,7376481,7717373,7891946,8029083,8077282,8842556,9796296,10020511,10020529,10020530,10725552,10813620,11928572,12049435,12309166,12416134,12459260,12518697,15306289,17124032
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-Guzzis-are-ace-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN
0009243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0Pete0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101edwin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13ellezee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
이슬 오,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
인호 한,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0
정훈 신,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0


In [9]:
rating_movies = pd.merge(ratings, movies, on='id')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('star_x', index='user', columns='name')

# NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

name,63 스카이 아트,63시티,N 서울 타워,가로수길,경복궁,경희궁,경희대학교 서울캠퍼스,관악산,광장시장,광화문,광화문 광장,국립 서울 현충원,국립중앙박물관,국립현대미술관 서울관,길상사,낙산 공공 미술,낙산공원,남대문시장,남산 공원,남산 케이블카,남산골 한옥 마을,노량진 수산물 도매시장,대한민국 역사 박물관,더 스파 그랜드 하얏트 서울,덕수궁,동대문 (흥인지문),동대문 디자인 플라자,동대문 역사 문화 공원,두물머리,드래곤힐스파,디큐브시브,롯데백화점 본점,롯데아울렛 서울역점,롯데월드,롯데월드타워&몰,마리오 아울렛,명동 성당,명동 쇼핑 거리,문화역서울 284,반포대교,...,선릉과 정릉,선유도 공원,세븐럭카지노 강북힐튼점,세종 문화 회관,세종대왕 동상,숭례문,신세계 백화점 본점,신촌,실로암 불가마 사우나,아차산,여의도 공원,여의도 한강공원,예술의전당,올림픽 공원,을왕리 해변,응봉산,이태원,이화 벽화마을,이화여자대학교,인사동,인왕산,잠실 야구 경기장,조계사,종묘,창경궁,창덕궁,청계천,청와대,충무공 이순신 동상,코엑스 아쿠아리움,쿠킹 난타,키자니아 서울,타임스퀘어,트릭아이뮤지엄 서울,풋볼팬타지움,하늘공원,하이서울페스티벌,한강 공원,한국 전쟁 기념관,홍대앞 거리
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-Guzzis-are-ace-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0009243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0Pete0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

user,-Guzzis-are-ace-,0009243,0Pete0,101edwin,13ellezee,1Ming,1matthias1962,22_L_C,22jh49,247Carpe_Diem,259darrent,2824,2hockey,30jieunl,398Suze63,3lfy-da,400seunghyeh,411MarianH,429Happy,493Ron,4K4K,54fg685,598saml,655Denis,950betty,A J,A Y,A. Buddy C,ADTU,AH1956,AR_Ismail90,ASHTAILORSAMUI,ASRPasig_Philippines,ATR310813,Aadhes,Aaron C,Aastha M,Abbydelight,Abdullmajeed A,Abhishek K,...,yusufahi01583,yyunz,zaidhill,zam78,zbasna,zoemanchester99,~mrs.htgalung~,Ирина Ц,Тума S,ᄋᄌᄋ ᄋ,ㅇㅅㅁ,蒲*宇 E,경호 박,기현 김,김용수,김환열,나라 이,동아 신,멀로 로,미송 김,백호50,범준 김,비니비니 J,성욱 황,수범 임,슈냥쓰,여행 다,연 이,연정 임,연준 성,유경 정,유영 김,은영 장,은주 박,의석 정,이슬 오,인호 한,정훈 신,제민 김,헌영 박
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63 스카이 아트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63시티,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N 서울 타워,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
ratings_matrix_T.shape

(121, 2892)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(121, 121)


name,63 스카이 아트,63시티,N 서울 타워,가로수길,경복궁,경희궁,경희대학교 서울캠퍼스,관악산,광장시장,광화문,광화문 광장,국립 서울 현충원,국립중앙박물관,국립현대미술관 서울관,길상사,낙산 공공 미술,낙산공원,남대문시장,남산 공원,남산 케이블카,남산골 한옥 마을,노량진 수산물 도매시장,대한민국 역사 박물관,더 스파 그랜드 하얏트 서울,덕수궁,동대문 (흥인지문),동대문 디자인 플라자,동대문 역사 문화 공원,두물머리,드래곤힐스파,디큐브시브,롯데백화점 본점,롯데아울렛 서울역점,롯데월드,롯데월드타워&몰,마리오 아울렛,명동 성당,명동 쇼핑 거리,문화역서울 284,반포대교,...,선릉과 정릉,선유도 공원,세븐럭카지노 강북힐튼점,세종 문화 회관,세종대왕 동상,숭례문,신세계 백화점 본점,신촌,실로암 불가마 사우나,아차산,여의도 공원,여의도 한강공원,예술의전당,올림픽 공원,을왕리 해변,응봉산,이태원,이화 벽화마을,이화여자대학교,인사동,인왕산,잠실 야구 경기장,조계사,종묘,창경궁,창덕궁,청계천,청와대,충무공 이순신 동상,코엑스 아쿠아리움,쿠킹 난타,키자니아 서울,타임스퀘어,트릭아이뮤지엄 서울,풋볼팬타지움,하늘공원,하이서울페스티벌,한강 공원,한국 전쟁 기념관,홍대앞 거리
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63 스카이 아트,1.000000,0.079153,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.089105,0.043821,0.00000,0.000000,0.0,0.0,0.0,0.035424,0.0,0.000000,0.000000,0.000000,0.044546,0.0,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.070001,0.000000,...,0.036865,0.0,0.0,0.0,0.000000,0.000000,0.0,0.076961,0.000000,0.0,0.000000,0.000000,0.046382,0.000000,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.016944,0.000000,0.0,0.036999,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000
63시티,0.079153,1.000000,0.0,0.000000,0.000000,0.040954,0.013719,0.0,0.0,0.000000,0.022485,0.036525,0.018329,0.0,0.0,0.067289,0.033092,0.00000,0.000000,0.0,0.0,0.0,0.038789,0.0,0.000000,0.035037,0.000000,0.051805,0.0,0.015971,0.0,0.0,0.017473,0.00000,0.000000,0.0,0.000000,0.011479,0.068280,0.025376,...,0.027839,0.0,0.0,0.0,0.000000,0.000000,0.0,0.093556,0.000000,0.0,0.011456,0.020256,0.035026,0.020731,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.04557,0.0,0.0,0.000000,0.000000,0.019279,0.0,0.036889,0.000000,0.000000,0.0,0.074041,0.000000,0.0,0.000000,0.0,0.039441,0.000000,0.011196
N 서울 타워,0.000000,0.000000,1.0,0.027293,0.102616,0.022172,0.000000,0.0,0.0,0.017849,0.014266,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.05787,0.028445,0.0,0.0,0.0,0.000000,0.0,0.013231,0.000000,0.040159,0.000000,0.0,0.037530,0.0,0.0,0.000000,0.04761,0.010008,0.0,0.042133,0.237378,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.019355,0.014407,0.0,0.000000,0.023435,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.010712,0.01459,0.021178,0.110376,0.018431,0.00000,0.0,0.0,0.038902,0.082698,0.078281,0.0,0.019971,0.018040,0.043332,0.0,0.000000,0.051607,0.0,0.021809,0.0,0.021526,0.044712,0.047354


In [13]:
item_sim_df["63시티"].sort_values(ascending=False)[:6]

name
63시티         1.000000
서울 대학로       0.096268
신촌           0.093556
63 스카이 아트    0.079153
타임스퀘어        0.074041
문화역서울 284    0.068280
Name: 63시티, dtype: float64

In [14]:
item_sim_df["광화문"].sort_values(ascending=False)[1:6]

name
광화문 광장     0.188131
창경궁        0.176222
숭례문        0.117011
세종대왕 동상    0.114394
종묘         0.113540
Name: 광화문, dtype: float64

In [0]:
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred


In [16]:
ratings_matrix.head(3)

name,63 스카이 아트,63시티,N 서울 타워,가로수길,경복궁,경희궁,경희대학교 서울캠퍼스,관악산,광장시장,광화문,광화문 광장,국립 서울 현충원,국립중앙박물관,국립현대미술관 서울관,길상사,낙산 공공 미술,낙산공원,남대문시장,남산 공원,남산 케이블카,남산골 한옥 마을,노량진 수산물 도매시장,대한민국 역사 박물관,더 스파 그랜드 하얏트 서울,덕수궁,동대문 (흥인지문),동대문 디자인 플라자,동대문 역사 문화 공원,두물머리,드래곤힐스파,디큐브시브,롯데백화점 본점,롯데아울렛 서울역점,롯데월드,롯데월드타워&몰,마리오 아울렛,명동 성당,명동 쇼핑 거리,문화역서울 284,반포대교,...,선릉과 정릉,선유도 공원,세븐럭카지노 강북힐튼점,세종 문화 회관,세종대왕 동상,숭례문,신세계 백화점 본점,신촌,실로암 불가마 사우나,아차산,여의도 공원,여의도 한강공원,예술의전당,올림픽 공원,을왕리 해변,응봉산,이태원,이화 벽화마을,이화여자대학교,인사동,인왕산,잠실 야구 경기장,조계사,종묘,창경궁,창덕궁,청계천,청와대,충무공 이순신 동상,코엑스 아쿠아리움,쿠킹 난타,키자니아 서울,타임스퀘어,트릭아이뮤지엄 서울,풋볼팬타지움,하늘공원,하이서울페스티벌,한강 공원,한국 전쟁 기념관,홍대앞 거리
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-Guzzis-are-ace-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0009243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0Pete0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
import numpy as np
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(3)

(2892, 121)


name,63 스카이 아트,63시티,N 서울 타워,가로수길,경복궁,경희궁,경희대학교 서울캠퍼스,관악산,광장시장,광화문,광화문 광장,국립 서울 현충원,국립중앙박물관,국립현대미술관 서울관,길상사,낙산 공공 미술,낙산공원,남대문시장,남산 공원,남산 케이블카,남산골 한옥 마을,노량진 수산물 도매시장,대한민국 역사 박물관,더 스파 그랜드 하얏트 서울,덕수궁,동대문 (흥인지문),동대문 디자인 플라자,동대문 역사 문화 공원,두물머리,드래곤힐스파,디큐브시브,롯데백화점 본점,롯데아울렛 서울역점,롯데월드,롯데월드타워&몰,마리오 아울렛,명동 성당,명동 쇼핑 거리,문화역서울 284,반포대교,...,선릉과 정릉,선유도 공원,세븐럭카지노 강북힐튼점,세종 문화 회관,세종대왕 동상,숭례문,신세계 백화점 본점,신촌,실로암 불가마 사우나,아차산,여의도 공원,여의도 한강공원,예술의전당,올림픽 공원,을왕리 해변,응봉산,이태원,이화 벽화마을,이화여자대학교,인사동,인왕산,잠실 야구 경기장,조계사,종묘,창경궁,창덕궁,청계천,청와대,충무공 이순신 동상,코엑스 아쿠아리움,쿠킹 난타,키자니아 서울,타임스퀘어,트릭아이뮤지엄 서울,풋볼팬타지움,하늘공원,하이서울페스티벌,한강 공원,한국 전쟁 기념관,홍대앞 거리
user,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-Guzzis-are-ace-,1.901723,0.000000,1.167887,0.589657,0.338649,0.746542,0.000000,0.000000,0.0,0.391752,0.391768,0.336171,0.295226,0.0,0.000000,0.0,0.191128,0.000000,0.712581,0.605687,0.195335,0.0,0.000000,0.0,1.052834,0.0,0.225656,0.188403,0.391566,0.315178,0.0,0.0,0.227089,0.985025,0.420609,0.0,0.652961,0.478795,0.755051,0.000000,...,0.181146,0.699989,0.0,0.000000,0.0,0.369354,0.206294,0.791155,0.336808,0.457003,0.259935,0.759938,0.0,0.633006,0.0,0.452341,0.212886,0.202714,0.0,0.304735,0.269359,0.271068,0.236058,0.000000,1.031637,0.264061,0.0,0.000000,0.339486,0.0,0.0,0.0,0.0,1.432822,0.0,0.521469,0.0,0.229995,0.0,0.0
0009243,0.000000,0.000000,0.334155,0.000000,0.000000,0.885681,0.997167,3.618696,0.0,0.000000,0.268120,0.978413,0.000000,0.0,0.000000,0.0,0.704611,0.275455,0.000000,0.382185,0.315842,0.0,0.291913,0.0,0.196023,0.0,0.218921,0.243706,0.633132,0.509619,0.0,0.0,0.367186,0.000000,0.000000,0.0,0.000000,0.000000,0.201908,0.698321,...,0.527218,1.227406,0.0,1.353286,0.0,0.397668,0.333561,0.635176,0.544593,1.330089,0.420295,0.351075,0.0,0.314930,0.0,0.731401,0.344220,0.327773,0.0,0.315349,15.178322,0.438296,0.000000,0.454477,0.000000,0.560180,0.0,0.475427,0.145027,0.0,0.0,0.0,0.0,0.000000,0.0,0.374744,0.0,0.371884,0.0,0.0
0Pete0,0.000000,0.608257,0.000000,0.000000,0.000000,0.364678,0.000000,0.000000,0.0,0.000000,0.264956,0.537147,0.000000,0.0,2.645921,0.0,0.305392,0.000000,0.000000,0.377674,0.312114,0.0,0.000000,0.0,0.193709,0.0,0.000000,0.000000,0.625658,0.503604,0.0,0.0,0.362852,0.000000,0.000000,0.0,0.000000,0.000000,0.532065,0.000000,...,0.289441,0.497098,0.0,0.000000,0.0,0.218319,0.329624,0.313839,0.538165,0.730216,0.415334,0.346931,0.0,0.311213,0.0,0.722768,0.340157,0.323904,0.0,0.000000,0.430392,0.433122,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.370321,0.0,0.367494,0.0,0.0


In [18]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

아이템 기반 모든 인접 이웃 MSE:  713.4950288987005


In [0]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred


In [20]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if sys.path[0] == '':


아이템 기반 인접 TOP-20 이웃 MSE:  477.8532647097816


In [21]:
user_rating_id = ratings_matrix.iloc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

name
실로암 불가마 사우나     40.0
동대문 역사 문화 공원    30.0
Name: 247Carpe_Diem, dtype: float64

In [0]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list


In [23]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않은 관광지 추출   
unseen_list = get_unseen_movies(ratings_matrix, '0009243')

# 아이템 기반의 인접 이웃 협업 필터링으로 관광지 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, '0009243', unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
name,
관악산,3.618696
선유도 공원,1.991982
아차산,1.906762
서울식물원,1.903271
국립 서울 현충원,1.504072
경희궁,1.378646
세종 문화 회관,1.365701
서울약령시장,1.292116
서울대공원,1.167287


In [0]:
def result(x):
  # 사용자가 관람하지 않은 관광지 추출   
  unseen_list = get_unseen_movies(ratings_matrix, x)

  # 아이템 기반의 인접 이웃 협업 필터링으로 관광지 추천 
  recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, x, unseen_list, top_n=1)

  # 평점 데이타를 DataFrame으로 생성. 
  recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
  return recomm_movies

In [0]:
def resultX(x):
  # 사용자가 관람하지 않은 관광지 추출   
  unseen_list = get_unseen_movies(ratings_matrix, x)

  # 아이템 기반의 인접 이웃 협업 필터링으로 관광지 추천 
  recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, x, unseen_list, top_n=10)

  # 평점 데이타를 DataFrame으로 생성. 
  recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
  return recomm_movies

In [0]:
def choice(x):
  import pandas as pd
  mname=list(movies['name']) ## 모든 관광지명 리스트
  rating_movies = pd.merge(ratings, movies, on='id')

  # columns='title' 로 title 컬럼으로 pivot 수행. 
  ratings_matrix = rating_movies.pivot_table('star_x', index='user', columns='name')
  # NaN 값을 모두 0 으로 변환
  ratings_matrix = ratings_matrix.fillna(0)
  ratings_matrix2 = ratings_matrix
  

  Rx=result(x)
  df_Rx = pd.DataFrame(Rx)
  df_Rx = df_Rx.reset_index()
  df_r1 = list(df_Rx['name'])[0] ## 해당 아이디 x 의 추천 탑 1 결과
  
  r1index = mname.index(df_r1) ## 추천 관광지명의 인덱스 
  mid = movies['id'][r1index] ## 해당 관광지 명의 관광지 id 
  rstar = list(movies[movies['id']==mid]['star'])[0] ## 해당 관관지 명의 별점 
  ratings_matrix2[df_r1].loc[x]=rstar
  ratings_matrix_T2 = ratings_matrix2.transpose()

  from sklearn.metrics.pairwise import cosine_similarity

  item_sim2 = cosine_similarity(ratings_matrix_T2, ratings_matrix_T2)

  # cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
  item_sim_df2 = pd.DataFrame(data=item_sim2, index=ratings_matrix2.columns,
                          columns=ratings_matrix2.columns)
  
  ratings_pred = predict_rating_topsim(ratings_matrix2.values , item_sim_df2.values, n=20)
  print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))
  ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
  re = resultX(x)
  return re




In [27]:
choice('chocho0258')

KeyError: ignored

In [38]:
result('swk')

,pred_score
name,
명동 쇼핑 거리,5.712855


In [37]:
rating_movies[rating_movies['user']=='choco0258']

,user,id,star_x,name,star_y,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
144,swk,324888,50,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/historical/great/in...


['63 스카이 아트',
 '63시티',
 'N 서울 타워',
 '가로수길',
 '경희궁',
 '경희대학교 서울캠퍼스',
 '관악산',
 '광장시장',
 '광화문',
 '광화문 광장',
 '국립 서울 현충원',
 '국립중앙박물관',
 '국립현대미술관 서울관',
 '길상사',
 '낙산 공공 미술',
 '낙산공원',
 '남대문시장',
 '남산 공원',
 '남산 케이블카',
 '남산골 한옥 마을',
 '노량진 수산물 도매시장',
 '대한민국 역사 박물관',
 '더 스파 그랜드 하얏트 서울',
 '덕수궁',
 '동대문 (흥인지문)',
 '동대문 디자인 플라자',
 '동대문 역사 문화 공원',
 '두물머리',
 '드래곤힐스파',
 '디큐브시브',
 '롯데백화점 본점',
 '롯데아울렛 서울역점',
 '롯데월드',
 '롯데월드타워&몰',
 '마리오 아울렛',
 '명동 성당',
 '명동 쇼핑 거리',
 '문화역서울 284',
 '반포대교',
 '봉은사',
 '북악산 서울 성곽',
 '북촌 한옥마을',
 '북한산 국립공원',
 '삼성미술관 리움',
 '삼청각',
 '삼청공원',
 '삼청동길',
 '서울 광장',
 '서울 극장',
 '서울 대학로',
 '서울 미술관',
 '서울 시립 미술관',
 '서울 시청',
 '서울 애니메이션 센터',
 '서울 어린이 대공원',
 '서울 에스케이프 룸 홍대점',
 '서울 역사 박물관',
 '서울 올림픽 기념관',
 '서울 월드컵 경기장 (상암 월드컵 경기장)',
 '서울 한양 도성',
 '서울글로벌문화체험센터',
 '서울대공원',
 '서울대학교 규장각',
 '서울대학교 미술관',
 '서울대학교병원 의학박물관',
 '서울도서관',
 '서울랜드',
 '서울로7017',
 '서울불꽃축제',
 '서울빛초롱축제',
 '서울성공회성당',
 '서울숲',
 '서울스카이',
 '서울식물원',
 '서울약령시장',
 '서울이스케이프룸 - 강남1호점',
 '서울이스케이프룸 - 강남2호점',
 '서울이스케이프룸 홍대2호점',
 '서울중앙성원',
 '서울풍물시장',